# ACDtools dev sandbox 

#### Date: 11 October, 2024

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

# Install ACDtools locally

In [1]:
# this needs to be set via a custom edit per user at the moment
!pip install --user -e /g/data/es60/users/thomas_moore/code/ACDtools

Obtaining file:///g/data/es60/users/thomas_moore/code/ACDtools
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ACDtools (pyproject.toml) ... done
  Created wheel for ACDtools: filename=ACDtools-0.1-0.editable-py3-none-any.whl size=3511 sha256=862276d6de04131aba4ed6bd86ab4067495cac240225dc15b26d193cc00bf755
  Stored in directory: /jobfs/130143991.gadi-pbs/pip-ephem-wheel-cache-09t75819/wheels/b6/a3/f2/6ce45fbdc116ad50e421d6a11cb060cc796e867501807af446
Successfully built ACDtools
  Attempting uninstall: ACDtools
    Found existing installation: ACDtools 0.1
    Uninstalling ACDtools-0.1:
      Successfully uninstalled ACDtools-0.1


In [2]:
# Enable autoreload in the notebook
%load_ext autoreload
%autoreload 1 
%aimport ACDtools.util
%aimport ACDtools.ard
# Importing from your local package util.py
from ACDtools.util import test_function
from ACDtools.util import detect_compute_platform
from ACDtools.util import load_config
from ACDtools.util import start_dask_cluster_from_config
from ACDtools.util import report_esm_unique
from ACDtools.util import var_name_info
from ACDtools.util import list_catalog_query_kwargs
from ACDtools.util import load_cmip6_fs38_datastore
from ACDtools.util import show_methods
from ACDtools.util import load_cmip6_CLEX_datastore
from ACDtools.util import remove_encoding
# ard.py
from ACDtools.ard import load_ACCESS_ESM_ensemble
from ACDtools.ard import load_ACCESS_ESM
from ACDtools.ard import find_chunking_info
from ACDtools.ard import save_n_drop_multidim_lat_lon

# Notebook settings

### filter warnings

In [3]:
import warnings
warnings.filterwarnings("ignore") # Suppress warnings

# Dask cluster from config
`client, cluster = start_dask_cluster_from_config('netcdf_work')`
<br>OR<br>
`client, cluster = start_dask_cluster_from_config('zarr_work')`

In [4]:
client, cluster = start_dask_cluster_from_config('netcdf_work')

Cluster started with 28 workers.
Dashboard available at: /proxy/38759/status


2024-12-05 12:58:52,269 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,306 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,312 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,321 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,339 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,365 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,368 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,372 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,376 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,381 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,671 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,674 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,679 - distributed.nanny - WARNING - Restarting worker
2024-12-05 12:58:52,685 - distributed.

# Issue: write function to load ACCESS-ESM1.5 data object using intake catalogs at NCI
- https://github.com/Thomas-Moore-Creative/ACDtools/issues/1

## utilise CMIP6 data catalogs for NCI holdings

##### Information on climate data catalogs across Australian HPC

**ACCESS-NRI** https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/how.html <br>
**NCI** https://opus.nci.org.au/pages/viewpage.action?pageId=213713098


##### $\bigstar$ Get inspiration from ACCESS-NRI intake catalog docs: ACCESS-ESM1-5 CMIP6 example
https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/quickstart.html

## import packages

In [5]:
import intake
import xarray as xr
import numpy as np
import gc
import json

### import the ACCESS-NRI catalog

In [ ]:
catalog = intake.cat.access_nri

### (1) "I know I want Australian CMIP6 data - so that's fs38 and I need access to that NCI project"

In [ ]:
cmip6_fs38_datastore = catalog.search(name='cmip6_fs38').to_source()

### (2) "what are the realms covered by cmip6_fs38?"

In [ ]:
report_esm_unique(cmip6_fs38_datastore,keep_list=['realm'])

### (3) I want to see what variables, over what frequencies, are available in both the 'ocean' & 'oceanBgchem' realms

In [ ]:
cmip6_fs38_ocean_datastore = cmip6_fs38_datastore.search(realm=['ocean','ocnBgchem'])

In [ ]:
[sorted_unique_dict, table_data] = report_esm_unique(cmip6_fs38_ocean_datastore,return_results=True)

## what is the long name of a particular variable?

In [ ]:
var_name_info(cmip6_fs38_ocean_datastore,'intpp')

## filter catalog for final ACCESS-ESM1.5 dataset

In [ ]:
final_search = cmip6_fs38_ocean_datastore.search(file_type='l',
                    variable_id='intpp',source_id='ACCESS-ESM1-5',experiment_id='historical')

In [ ]:
report_esm_unique(final_search)

## what is the chunking of the files in this final_search catalog?

In [ ]:
final_search.df['path'].iloc[0]

In [ ]:
find_chunking_info(final_search,'intpp',return_results=False)

## load without specifying any chunking

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(final_search)

In [ ]:
ds_ESM15_esorted

#### One still needs to know what dimensions (1, 300, 360 ; ) refers to and something about MB size per chunk to set the time to 220 . . . these rules of thumb should be in the yaml settings file until much more complicated heuristics could be coded

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(final_search,chunking_settings={'chunks':{'member':1,'time':220,'j':300,'i':360}})

In [ ]:
ds_ESM15_esorted

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(final_search,chunking_key='ACCESS_ESM15_2D')

In [ ]:
ds_ESM15_esorted

In [ ]:
ds_ESM15_esorted.isel(member=0).mean('time').intpp.plot()

## 3D dataset?

In [ ]:
thetao_search = cmip6_fs38_ocean_datastore.search(file_type='l',
                    variable_id='thetao',source_id='ACCESS-ESM1-5',experiment_id='historical')

In [ ]:
report_esm_unique(thetao_search)

In [ ]:
find_chunking_info(thetao_search,'thetao',return_results=False)

In [ ]:
find_chunking_info(thetao_search,'thetao',return_results=True)

In [ ]:
xr.open_mfdataset('/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r3i1p1f1/Omon/thetao/gn/v20191203/thetao_Omon_ACCESS-ESM1-5_historical_r3i1p1f1_gn_189001-189912.nc')

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(thetao_search)

In [ ]:
ds_ESM15_esorted

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(thetao_search,chunking_key='ACCESS_ESM15_3D')

In [ ]:
ds_ESM15_esorted

# let's use the tools as they exist to try to start the workflow

## I want Australian CMIP6 data

In [ ]:
cmip6_fs38_datastore = load_cmip6_CLEX_datastore()

In [ ]:
report_esm_unique(cmip6_fs38_datastore.search(**load_config()['catalog_search_query_dict']['ACCESS_ESM15']['CSEPTA']['CLEX_catalog']))

In [ ]:
CSEPTA_catalog = cmip6_fs38_datastore.search(**load_config()['catalog_search_query_dict']['ACCESS_ESM15']['CSEPTA']['CLEX_catalog'])

In [ ]:
CSEPTA_catalog

In [ ]:
show_methods(CSEPTA_intpp_catalog)

In [ ]:
report_esm_unique(CSEPTA_intpp_catalog)

In [ ]:
CSEPTA_intpp_catalog.unique()['path']

In [ ]:
search_dict = dict(experiment_id = 'historical',source_id = 'ACCESS-ESM1-5',variable_id = ['intpp'],realm = ['ocnBgchem'], frequency = 'mon',file_type='f')

In [ ]:
search = cmip6_fs38_datastore.search(**search_dict)
search

In [ ]:
search.unique()['path']

# let's repeat workflow with CLEX catalog

In [ ]:
cmip6_fs38_datastore = load_cmip6_CLEX_datastore()

In [ ]:
load_config()['catalog_search_query_dict']['ACCESS_ESM15']['CSEPTA']['CLEX_catalog']

In [ ]:
report_esm_unique(cmip6_fs38_datastore.search(**load_config()['catalog_search_query_dict']['ACCESS_ESM15']['CSEPTA']['CLEX_catalog']))

In [ ]:
find_chunking_info(cmip6_fs38_datastore.search(**load_config()['catalog_search_query_dict']['ACCESS_ESM15']['CSEPTA']['CLEX_catalog']),'thetao',return_results=True)

In [ ]:
search = cmip6_fs38_datastore.search(**load_config()['catalog_search_query_dict']['ACCESS_ESM15']['CSEPTA']['CLEX_catalog'])

In [ ]:
%%time
ds_ESM15_esorted = load_ACCESS_ESM_ensemble(search,use_cftime=True,chunking_key='ACCESS_ESM15_3D')

In [ ]:
ds_ESM15_esorted

In [ ]:
data = ds_ESM15_esorted.isel({'lev':0,'member':0})
data

In [ ]:
data.mean(dim=['i','j']).thetao.plot()

### piControl

In [ ]:
search = cmip6_fs38_datastore.search(**load_config()['catalog_search_query_dict']['ACCESS_ESM15']['CSEPTA']['CLEX_catalog'])

In [ ]:
%%time
ds_ESM15 = load_ACCESS_ESM(search,use_cftime=True,chunking_key='ACCESS_ESM15_3D')

In [ ]:
ds_ESM15